In [ ]:
import torch
import numpy as np
import phi.torch.flow as ptf

device = torch.device('cuda')

In [ ]:
BOUNDS = ptf.Box(x=1, y=1, z=1)
RESOLUTION = 100
DENSITY = ptf.CenteredGrid(
    values=0,
    extrapolation=ptf.extrapolation.ZERO_GRADIENT,
    bounds=BOUNDS,
    resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)
VELOCITY = ptf.StaggeredGrid(
    values=0,
    extrapolation=ptf.extrapolation.ZERO,
    bounds=BOUNDS,
    resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)
INFLOW = ptf.CenteredGrid(
    values=ptf.Sphere(center=ptf.tensor([0.5, 0, 0.5], ptf.channel(vector='x,y,z')), radius=0.05),
    extrapolation=ptf.extrapolation.ZERO,
    bounds=BOUNDS,
    resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)

In [ ]:
OBSTACLE = ptf.Obstacle(geometry=ptf.Sphere(center=ptf.tensor([0.5, 0.5, 0.5], ptf.channel(vector='x,y,z')), radius=0.2),
                        velocity=(0., 0., 0.),
                        angular_velocity=(0., 0., 0.))

In [ ]:
@ptf.jit_compile
def step(v, s, p, dt, inflow_rate, inflow, obs):
    s = ptf.advect.mac_cormack(s, v, dt) + inflow_rate * ptf.resample(inflow, to=s, soft=True)
    buoyancy = ptf.resample(s * (0, 0.1, 0), to=v)
    v = ptf.advect.semi_lagrangian(v, v, dt) + buoyancy * dt
    v, p = ptf.fluid.make_incompressible(v, (obs), ptf.Solve('CG', 1e-3, x0=p))
    return v, s, p

# @ptf.jit_compile_linear

In [ ]:
import tqdm

for _ in tqdm.trange(119):
    VELOCITY, DENSITY, _ = step(VELOCITY, DENSITY, None, 0.2, 0.4, INFLOW, OBSTACLE)